This notebook contains the details about the processing part done mostly by Hila. 

# Merging the data by chromosomes

In [ ]:
# Go to the directory with normalized data from previous step (3_normalize_data.ipynb)

cd .../450k_data/

In [ ]:
#Merging all the norm.part* files without headers, output stored in .../450k/RAP_output_per_chr/
first=1
for i in {1..22}; do for f in ./chr$i/output.chr$i.part*.norm.csv.gz; do if [ "$first" ]; then zcat "$f"; first=; else zcat "$f"| tail -n +2; fi; done >.../450k/RAP_output_per_chr/chr$i.all_parts.csv ; done

# Update samples

The file `.../450k/samples/ancestry/european_non_related_to_include.txt` is a combination of the output from script `pipeline/1_generate_unrelaed_samples.ipynb` (`./450k/samples/relatedness/related_samples_to_remove_final.txt`) and samples with non-european genetic ancestry (based on data-field `22006`).


The file `.../450k/samples/withdrawal_samples.txt` contains samples that need to be withdrew due to revoked consent. 

In [ ]:
#create an updated list of identifiers (rempve most updated withdrawals)
fgrep -vf .../450k/samples/withdrawal_samples.txt .../450k/samples/ancestry/european_non_related_to_include.txt > .../450k/samples/european_non_related_no_withdrawal_to_include.txt

In [ ]:
#create updated per-chr files with samples to include
for i in {1..22}; do awk -v FS="," 'FNR==NR{arr[$0];next} ($6 in arr)' .../450k/samples/european_non_related_no_withdrawal_to_include.txt .../450k/RAP_output_per_chr/chr$i.all_parts.csv> .../450k/RAP_output_per_chr/chr$i.all_parts_updated_samples.csv; done

In [ ]:
#Extracting the header randomly from one of the files:
zcat ./chr22/output.chr22.part0.norm.csv.gz|grep chrom > .../450k/RAP_output_per_chr/header.txt

In [ ]:
#Adding header to the individual merged files per chromosome:
for i in {1..22}; do cat .../450k/RAP_output_per_chr/header.txt  .../450k/RAP_output_per_chr/chr$i.all_parts_updated_samples.csv > .../450k/RAP_output_per_chr/chr$i.all_parts_final.csv; done

In [ ]:
#Removing un-needed files:
rm -f .../450k/RAP_output_per_chr/*all_parts.csv
rm -f .../450k/RAP_output_per_chr/*all_parts_updated_samples.csv

# Preparations for annotation

For the annotaion, we need to activate our inhouse research annotation tool.
These are the necessary preparations for that:

In [ ]:
#make job files for hcdiff (based on .../450k/scripts/vcfPos_forWebAnnot_new.py)
for i in {1..22}; do cp .../450k/scripts/vcfPos_forWebAnnot_new.py .../450k/RAP_output_per_chr/files_for_annotation/job_files/hcdiff_${i}.job ; done
cd .../450k/RAP_output_per_chr/files_for_annotation/job_files/
for f in hcdiff*.job ; do n=$(echo $f | cut -d"." -f1 |cut -d"_" -f2); sed 's:'number':'${n}':g' ${f} > hcdiff_${n}_ready.py; done

In [ ]:
#make job files for in house annotation (based on .../450k/scripts/annotate_in_house.sh)
for i in {1..22}; do cp .../450k/scripts/annotate_in_house.sh .../450k/RAP_output_per_chr/files_for_annotation/job_files/annotate_in_house_${i}.job ; done
cd .../450k/RAP_output_per_chr/files_for_annotation/job_files/
for f in annotate_in_house*.job ; do n=$(echo $f | cut -d"." -f1 |cut -d"_" -f4); sed 's:'number':'${n}':g' ${f} > annotate_in_house_${n}_ready.sh; done

In [ ]:
#make job files for running all preparations (based on .../450k/scripts/prepare_for_annotate.py)
for i in {1..22}; do cp .../450k/scripts/prepare_for_annotate.py .../450k/RAP_output_per_chr/files_for_annotation/job_files/prepare_for_annotate_${i}.job ; done
cd .../450k/RAP_output_per_chr/files_for_annotation/job_files/
for f in prepare_for_annotate_*.job ; do n=$(echo $f | cut -d"." -f1 |cut -d"_" -f4); sed 's:'number':'${n}':g' ${f} > prepare_for_annotate_${n}_ready.py; done

In [ ]:
#make sh job files for running all preparations files (based on .../450k/scripts/prepare_for_annotate.sh)
for i in {1..22}; do cp .../450k/scripts/prepare_for_annotate.sh .../450k/RAP_output_per_chr/files_for_annotation/job_files/prepare_for_annotate_${i}.sh ; done
cd .../450k/RAP_output_per_chr/files_for_annotation/job_files/
for f in prepare_for_annotate_*.sh ; do n=$(echo $f | cut -d"." -f1 |cut -d"_" -f4); sed 's:'number':'${n}':g' ${f} > prepare_for_annotate_${n}_ready.sh; done

In [ ]:
#run the .../450k/scripts/prepare_for_annotate_X_ready.sh files
cd .../450k/RAP_output_per_chr/files_for_annotation/job_files/
for i in {1..22}; do sbatch prepare_for_annotate_${i}_ready.sh; done


# Annotation

Annotation runs for each chromosome separately, using a template script

In [ ]:
#make job files for running all annotations (based on .../450k/scripts/annotate_pandas.py)
for i in {1..22}; do cp .../450k/scripts/annotate_pandas.py .../450k/plp_selection/job_files/annotate_pandas_${i}.job ; done
cd .../450k/plp_selection/job_files/
for f in annotate_pandas_*.job ; do n=$(echo $f | cut -d"." -f1 |cut -d"_" -f3); sed 's:'number':'${n}':g' ${f} > annotate_pandas_${n}_ready.py; done
for i in {1..22}; do python annotate_pandas_${i}_ready.py; done

# PLPs selection

PLPs selection runs for each chromosome separately, using a template script

In [ ]:
#make job files for running all plps selection (based on .../450k/scripts/select_plps_pandas.py)
for i in {1..22}; do cp .../450k/scripts/select_plps_pandas.py .../450k/plp_selection/job_files/select_plps_pandas_${i}.job ; done
cd .../450k/plp_selection/job_files/
for f in select_plps_pandas_*.job ; do n=$(echo $f | cut -d"." -f1 |cut -d"_" -f4); sed 's:'number':'${n}':g' ${f} > select_plps_pandas_${n}_ready.py; done
for i in {1..22}; do python select_plps_pandas_${i}_ready.py; done

In [ ]:
#edit presumable plps
#Notice there is a manual part inside this script
python .../450k/scripts/edit_plps.py

# PLPs statistics

In [ ]:
import pandas as pd

## file, that contains all PLPs found in the cohort
all_plps_file = (
    ".../450k/plp_selection/basic/new_gene_names/new_freq/new_relatedness/"
    "all_chr_total_presumable_plps_HFE_final_sorted.txt"
)

plps = pd.read_csv(all_plps_file, sep="\t")

In [ ]:
#conditions tier 1- "safe tier"
cond_clinvar = ((plps['clinvar'].str.contains("athogenic")) & (plps[~(plps['clinvar_stars']==" ")]['clinvar_stars']>=2))
cond_vkgl = (plps['vkgl']=="LP")     

tier_1_cond = (cond_clinvar | cond_vkgl)

In [ ]:
#conditions tier 2- "rare LOF"
cond_region = ((plps['region'].str.contains("CANONICAL")) | (plps['region']=="EXON_REGION"))
cond_synonymous = (~plps['synonymous'])
variant_types = ["stop_gained","frameshift","splice_donor","splice_acceptor"]
cond_variant_type = (plps['Protein Effect'].str.contains('|'.join(variant_types),na=False))

cond_lof = (cond_region & cond_synonymous & cond_variant_type)
cond_freq_var = (plps['gnomAD-E_AF']<1)

cond_clinvar_not = ~(plps['clinvar'].str.contains("enign") & (plps[~(plps['clinvar_stars']==" ")]['clinvar_stars']>=2))
cond_vkgl_not = ~(plps['vkgl']=="LB")

tier_2_cond = (cond_lof & cond_freq_var & cond_clinvar_not & cond_vkgl_not)


In [ ]:
plps['tier'] = "missense"

plps.loc[tier_2_cond, 'tier'] = 'rare LoF'
plps.loc[tier_1_cond, 'tier'] = 'ClinVar/VKGL'

plps['tier'].value_counts()